<span STYLE="font-size:150%"> 
    Segment microCT scans
</span>

Docker image: gnasello/slicer-env:2023-07-06 \
Latest update: 10 March 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Load libraries

In [ ]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [ ]:
# this cell is tagged 'parameters'
volume_file = 'microCT_volume/microCT_volume.nrrd'
output_dir_path = 'segmented_volumes'
segmentMask_file = 'segmented_volumes/ROI.seg.nrrd'
segments_greyvalues_file = 'segmented_volumes/segments_greyvalues.csv'

In [ ]:
# Parameters
volume_file = "/config/researcher_home/Documents/microCT/2025-12-17_GN009/111701/microCT_volume/microCT_volume_preview.nrrd"
output_dir_path = "/config/researcher_home/Documents/microCT/2025-12-17_GN009/111701/segmented_volumes"
segmentMask_file = "/config/researcher_home/Documents/microCT/2025-12-17_GN009/111701/segmented_volumes/ROI.seg.nrrd"
segments_greyvalues_file = "/config/researcher_home/Documents/microCT/2025-12-17_GN009/111701/segmented_volumes/segments_greyvalues.csv"


In [ ]:
path = Path(volume_file)

# Remove image numbering _0000, _0001 ...
filename_output = path.stem[:-4]

In [ ]:
masterVolumeNode = slicer.util.loadNodeFromFile(volume_file)

Print spacing

In [ ]:
## mm
masterVolumeNode.GetSpacing()

Make ```segmented_volumes``` folder

In [ ]:
output_directory = Path(output_dir_path)

output_directory.mkdir(parents=True, exist_ok=True)

In [ ]:
# Monitor Memory in Slicer
import psutil, os
print(psutil.Process(os.getpid()).memory_info().rss / (1024**3), "GB used")

# Create segmentationNode

## Create segmentation-related nodes

Create segmentation node

In [ ]:
segmentationNode = ps.segmentation.segmentationNode(name='Segmentation')

Create temporary segment editor to get access to effects

In [ ]:
segmentEditorWidget, segmentEditorNode = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode, 
                                                                             masterVolumeNode = masterVolumeNode)

## Load Segment Mask LabelmapNode

In [ ]:

# We attempt to load an external segmentation mask.
# If this file does not exist or cannot be loaded,
# the code will fall back to running without any mask.

try:
    # Try loading the segmentation mask from disk as a labelmap volume.
    # The 'labelmap=True' property tells Slicer this volume represents labels,
    # not grayscale intensities.
    segmentMask_labelmap = slicer.util.loadNodeFromFile(
        segmentMask_file,
        'VolumeFile',
        properties={'labelmap': True}
    )

    # Import the labelmap into the existing segmentation node.
    # This converts the labelmap into a proper Slicer segment.
    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(
        segmentMask_labelmap,
        segmentationNode
    )

    # Access the internal segmentation object
    segmentation = segmentationNode.GetSegmentation()

    # Assume the imported mask is the first segment
    segment = segmentation.GetNthSegment(0)

    # Rename the segment to a known, stable name
    # so we can reliably reference it later
    segment.SetName("SegmentMask")

    # Remove the temporary labelmap node from the scene
    # since it is no longer needed after import
    slicer.mrmlScene.RemoveNode(segmentMask_labelmap)

    # Retrieve the ID of the mask segment by name
    mask_id = segmentation.GetSegmentIdBySegmentName("SegmentMask")

    # Tell the Segment Editor to use this segment as a mask
    segmentEditorNode.SetMaskSegmentID(mask_id)

    # Mask mode 5 means:
    # "Editing is allowed only inside the selected mask segment"
    segmentEditorNode.SetMaskMode(5)

    print("External segmentation mask loaded. Segmentation will be restricted to ROI.")

except Exception as e:
    # If anything fails above (missing file, unreadable file, wrong format),
    # we catch the exception and continue execution.

    print(
        "No external segmentation mask found or mask could not be loaded. "
        "Proceeding without any mask.\nDetails:",
        e
    )

    # # Disable masking altogether
    # # This allows segmentation effects to operate everywhere in the volume.
    # segmentEditorNode.SetMaskMode(
    #     slicer.vtkMRMLSegmentEditorNode.EditAllowedEverywhere
    # )

    # # Explicitly clear any mask segment association
    # segmentEditorNode.SetMaskSegmentID(None)


# Thresholding

## Read file with thresholding values

In [ ]:
grey_df = pd.read_csv(segments_greyvalues_file)

segments_greyvalues = grey_df.to_dict('list')
segments_greyvalues

## Create segments by thresholding

In [ ]:
ps.segmentation.segments_by_thresholding(segments_greyvalues, 
                                         segmentationNode,
                                         segmentEditorNode,
                                         segmentEditorWidget)

Set segments color

In [ ]:
segments_color = {
    "Bone": (0.9450980392156862, 0.8392156862745098, 0.5686274509803921), # "Bone" color in Slicer
    }

segments_color

In [ ]:
ps.segmentation.set_segments_color(segments_color, segmentationNode)

In [ ]:
# Monitor Memory in Slicer
import psutil, os
print(psutil.Process(os.getpid()).memory_info().rss / (1024**3), "GB used")

# Operation on segments

## Manual fix of the segmentation

Sometimes it might be necessary to remove speckles at the image boundaries. If so, use the `scissor` tool in the `Segment Editor` before proceeding with the rest of the script. 

## Remove small islands

REMOVE_SMALL_ISLANDS operation from the [SegmentEditorIslandsEffect](https://github.com/Slicer/Slicer/blob/294ef47edbac2ccb194d5ee982a493696795cdc0/Modules/Loadable/Segmentations/EditorEffects/Python/SegmentEditorIslandsEffect.py#L402)

In [ ]:
# segment_name = 'Bone'
# minimum_size = 20 #number of voxels

In [ ]:
# ps.segmentation.remove_small_islands(minimum_size, 
#                                      segment_name, 
#                                      segmentEditorNode,
#                                      segmentEditorWidget)

## Smoothing thresholded segment

In [ ]:
segment_name = 'Bone'
gaussiaSD_mm = 0.001 #units are in millimiters

In [ ]:
ps.segmentation.gaussian_smoothing(gaussiaSD_mm, 
                                   segment_name, 
                                   segmentEditorNode, 
                                   segmentEditorWidget)

# Export segments

## As seg.nrrd file (labelmap node)

Export a selection of segments (identified by their names), from [slicer tutorial](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-labelmap-node-from-segmentation-node)

Bone LabelmapNode

In [ ]:
seg_name = 'Bone'

In [ ]:
mineralized_labelmap = ps.segmentation.individual_segment_to_labelmapNode(segmentName = seg_name,
                                                                          segmentationNode = segmentationNode,
                                                                          volumeNode = masterVolumeNode)

In [ ]:
filename_output = seg_name + '.seg.nrrd'

slicer.util.exportNode(mineralized_labelmap, output_directory / filename_output)

## Convert all segments to model nodes

Get closed surface representation of the segment, from [slicer scripting repository](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-nodes-warped-by-transform-sequence)

In [ ]:
segmentationNode.CreateClosedSurfaceRepresentation()

In [ ]:
shNode = slicer.mrmlScene.GetSubjectHierarchyNode()
exportFolderItemId = shNode.CreateFolderItem(shNode.GetSceneItemID(), "Segments")
slicer.modules.segmentations.logic().ExportAllSegmentsToModels(segmentationNode, exportFolderItemId)

## Decimate model node from segmentation node

Get `VTK Polydata` from model

In [ ]:
nodename = 'Bone'
modelNode = slicer.util.getNode(nodename)

Use pyvista to get faces and vertices of the `VTK PolyData` object

In [ ]:
import pyvista as pv

In [ ]:
model_pv = pv.wrap(modelNode.GetPolyData())
vertices = model_pv.points
faces = model_pv.regular_faces

In 3D Slicer, you can clear the scene using Python scripting by calling the Clear method on the application's MRML scene.

In [ ]:
#slicer.mrmlScene.Clear(0)

Import pyvista object to pymeshlab, which performs more efficient decimation than VTK

Check how to [import mesh from arrays in meshlab](https://pymeshlab.readthedocs.io/en/0.1.9/tutorials/import_mesh_from_arrays.html)

In [ ]:
import pymeshlab 
# create a new Mesh with the two arrays
model_ml = pymeshlab.Mesh(vertices, faces)

# create a new MeshSet
mesh_set = pymeshlab.MeshSet()

# add the mesh to the MeshSet
mesh_set.add_mesh(model_ml, "mesh")

print('input mesh has', model_ml.vertex_number(), 'vertex and', model_ml.face_number(), 'faces')

Decimate mesh with `pymeshlab`, from [stackoverflow](https://stackoverflow.com/questions/75169329/how-to-use-pymeshlab-to-reduce-number-of-faces-to-a-guaranteed-specific-number)

In [ ]:
decimation_factor = 0.2

#Target number of vertex
TARGET = round(model_ml.vertex_number() * decimation_factor)

#Reduce to TARGET. Sometimes will fall into TARGET-1
mesh_set.meshing_decimation_quadric_edge_collapse(targetfacenum=TARGET, preservenormal=True)

model_ml = mesh_set.current_mesh()
print('output mesh has', model_ml.vertex_number(), 'vertex and', model_ml.face_number(), 'faces')

Load decimated model to Slicer

In [ ]:
vertices = model_ml.vertex_matrix()
faces = model_ml.face_matrix()
model_decimated = pv.PolyData.from_regular_faces(vertices, faces)

decimatedNode = slicer.modules.models.logic().AddModel(model_decimated)

## Export decimated model from segmentation node

Export pymeshlab object to .stl file

In [ ]:
filename_output = 'Bone.stl'

slicer.util.exportNode(decimatedNode, output_directory / filename_output)